In [1]:
data = '''{
  "client": "Bavaria",
  "incoterm": "CFR",
  "cargo_types": [
    "20' DRY"
  ],
  "cargo_value": 0,
  "surcharges": {
    "Origen": {
      "20' DRY": {
        "cost": 98.94,
        "sale": 1000
      }
    },
    "Flete": {
      "20' DRY": {
        "cost": 332.8,
        "sale": 1234
      }
    }
  },
  "additional_surcharges": [
    {
      "concept": "Grid",
      "cost": 1000,
      "sale": 2000
    }
  ],
  "total_profit": 2802.26,
  "pol": "CTG",
  "pod": "CAUCEDO",
  "commodity": "FAK",
  "commercial": "Shadia Jaafar",
  "contract_id": "00072158"
}'''

import json

quotation_data = json.loads(data)



In [4]:
def user_data():
    user = "pricing@tradingsol.com"
    users = {
            "pricing@tradingsol.com": {
            "name": "Shadia Jaafar",
            "tel": "+57 12345678",
            "position": "Data Analyst",
            "email": "pricing@tradingsol.com"
        },
            }
    return users.get(user, {"name": "Desconocido", "position": "N/A", "tel": "N/A", "email": user})

In [3]:
import json
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import PyPDF2
from reportlab.platypus import Table, TableStyle
import streamlit as st

def create_overlay(data, overlay_path):

    commercial_data = user_data()
    c = canvas.Canvas(overlay_path, pagesize=letter)

    c.drawString(508, 688, f"{data.get('quotation', '')}")
    c.drawString(460, 675, f"{data.get('date', '')}")
    c.drawString(475, 660, f"{data.get('validity', '')}")

    c.drawString(90, 570, f"{commercial_data.get('name', '')}")
    c.drawString(90, 555, f"{commercial_data.get('position', '')}")
    c.drawString(90, 540, f"{commercial_data.get('tel', '')}")
    c.drawString(90, 525, f"{commercial_data.get('email', '')}")

    c.drawString(160, 470, f"MARITIME")
    c.drawString(160, 457, f"{data.get('incoterm', '')}")
    c.drawString(160, 444, f"{data.get('pol', '').upper()} - {data.get('pod', '').upper()}")

    c.setFont("Helvetica-Bold", 11) 
    c.drawString(400, 588, f"{data.get('client', '').upper()}")

    table_data = []
    total_cost = 0.0

    # Procesar los surcharges (cada clave es una categoría, por ejemplo "Origen" o "Flete")
    surcharges = data.get("surcharges", {})
    for category, details in surcharges.items():
        for container, values in details.items():
            cost = values.get("cost", 0)
            sale = values.get("sale", 0)
            total_cost += cost
            row = [
                category,                   # Concepto
                "USD",                      # Moneda
                container,                  # Contenedor
                f"${sale}"                  # Venta (con el signo $)
            ]
            table_data.append(row)

    # Procesar los additional_surcharges
    for additional in data.get("additional_surcharges", []):
        cost = additional.get("cost", 0)
        sale = additional.get("sale", 0)
        total_cost += cost
        row = [
            additional.get("concept", ""),  # Concepto
            "USD",                          # Moneda
            "",                             # Contenedor (no aplica)
            f"${sale}",                     # Costo (con el signo $)
        ]
        table_data.append(row)
    
    col_widths = [80, 140, 120, 85]

    table = Table(table_data, colWidths=col_widths)

    style = TableStyle([
        ('FONTNAME', (0,0), (-1,-1), 'Helvetica'),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('FONTSIZE', (0,0), (-1,-1), 10),
        ('TOPPADDING', (0,0), (-1,-1), 3),
        ('BOTTOMPADDING', (0,0), (-1,-1), 5),
    ])
    table.setStyle(style)

    x = 100
    y = 388
    table_width, table_height = table.wrapOn(c, 0, 0)
    table.drawOn(c, x, y - table_height)
    c.drawString(400, 270, f"${total_cost}")

    additional_info = ( 
        f"Transit Time: {data.get("Details", {}).get("Transit Time", "N/A")} days \n"
        f"Route: {data.get("Details", {}).get("Route", "N/A")} \n"
        f"Free Days in Origin: {data.get("Details", {}).get("Free Days in Origin", "N/A")} \n"
        f"Free Days in Destination: {data.get("Details", {}).get("Free Days in Destination", "N/A")} \n"
        f"Notes: {data.get("Details", {}).get("Notes", " ")}"
    )

    c.drawString(88, 200, additional_info)

    c.save()

def merge_pdfs(template_path, overlay_path, output_path):
    template_pdf = PyPDF2.PdfReader(template_path)
    overlay_pdf = PyPDF2.PdfReader(overlay_path)
    output = PyPDF2.PdfWriter()

    for page_number in range(len(template_pdf.pages)):
        template_page = template_pdf.pages[page_number]
        if page_number < len(overlay_pdf.pages):
            overlay_page = overlay_pdf.pages[page_number]
            template_page.merge_page(overlay_page)
        output.add_page(template_page)
    
    with open(output_path, "wb") as f_out:
        output.write(f_out)

def generate_quotation(data, template_path="plantilla_cotizacion (1).pdf", output_path="quotation.pdf", overlay_path="overlay.pdf"):
    create_overlay(data, overlay_path)
    merge_pdfs(template_path, overlay_path, output_path)
    return output_path

pdf_filename = generate_quotation(quotation_data)

with open(pdf_filename, "rb") as f:
    pdf_bytes = f.read()

st.download_button(
    label="Descargar Cotización",
    data=pdf_bytes,
    file_name="quotation.pdf",
    mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
)

2025-03-11 17:23:08.565 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 17:23:08.565 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 17:23:08.565 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 17:23:08.565 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 17:23:08.566 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-11 17:23:08.622 
  command:

    stre

False

In [ ]:
table_data = []
    for detail in data.get("details", []):
        row = [
            detail.get('concept', ''),
            detail.get('currency', ''),
            detail.get('container', ''),
            detail.get('rate', '')
        ]
        table_data.append(row)

    col_widths = [80, 140, 120, 85]

    table = Table(table_data, colWidths=col_widths)

    style = TableStyle([
        ('FONTNAME', (0,0), (-1,-1), 'Helvetica'),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('VALIGN', (0,0), (-1,-1), 'MIDDLE'),
        ('FONTSIZE', (0,0), (-1,-1), 10),
        ('TOPPADDING', (0,0), (-1,-1), 3),
        ('BOTTOMPADDING', (0,0), (-1,-1), 5),
    ])
    table.setStyle(style)

    x = 100
    y = 388
    table_width, table_height = table.wrapOn(c, 0, 0)
    table.drawOn(c, x, y - table_height)